## Install FaceNet

**With pip:**
`!sudo pip install facenet-pytorch`

**or clone this repo, removing the '-' to allow python imports:**

`!sudo git clone https://github.com/timesler/facenet-pytorch.git facenet_pytorch`

**or use a docker container (see https://github.com/timesler/docker-jupyter-dl-gpu):**

`!sudo docker run -it --rm timesler/jupyter-dl-gpu pip install facenet-pytorch && ipython`

## Get data
`!sudo wget https://download.openmmlab.com/datasets/movienet/poster4M.img_meta.v1.json`

In [ ]:
import json
import shutil
from pathlib import Path

from imdb import IMDb
import requests

In [ ]:
img_meta = json.load(open('poster4M.img_meta.v1.json'))
ia = IMDb()

p = Path.cwd() / 'data' / 'train_data'
shutil.rmtree(p, ignore_errors=True)
p.mkdir(parents=True, exist_ok=True)

i = 1
subsample = {}
for image in img_meta:
    if (img_meta[image]['type'] == 'event'    # Download only publicity type ~360K
            and len(img_meta[image]['cast']) == 1):    # Only one person in photo
        name_id = img_meta[image]['cast'][0][2:]
        name = ia.get_person(name_id)['name']
        url = img_meta[image]['url']

        r = requests.get(url)
        img = r.content
        if img == b'Not Found':
            continue

        folder = p / name
        folder.mkdir(parents=True, exist_ok=True)
        filepath = folder / f'{image}.jpg'
        with filepath.open('wb') as out_file:
            out_file.write(img)
        i += 1
        subsample[image] = img_meta[image]

    if i == 1000:
        break

json.dump(subsample, open('subsample10K_meta.json', 'w'), indent=6)

## Train test val split

In [13]:
p = Path.cwd() / 'data' / 'train_data'
test_path = Path.cwd() / 'data' / 'test_data'
test_path.mkdir(parents=True, exist_ok=True)
val_path = Path.cwd() / 'data' / 'val_data'
val_path.mkdir(parents=True, exist_ok=True)

i = 0
for actor in p.iterdir():
    images = list(actor.iterdir())
    if len(images) > 1:    # Half in val and half in test
        if i < 100:
            print(f'val for {actor.stem}')

            actor_folder = val_path / actor.stem
            actor_folder.mkdir(parents=True, exist_ok=True)
            for image in images[1:]:    # Leave first photo in train
                image.rename(actor_folder / image.name)
        else:
            print(f'test for {actor.stem}')

            actor_folder = test_path / actor.stem
            actor_folder.mkdir(parents=True, exist_ok=True)
            for image in images[1:]:
                image.rename(actor_folder / image.name)
        i += 1

val for Emma Roberts
val for Mila Kunis
val for Andrew Garfield
val for Martin Landau
val for Jennifer Lawrence
val for Rob Lowe
val for Ashley Greene
val for Gwendoline Christie
val for Marg Helgenberger
val for Ryan Seacrest
val for Sam Rockwell
val for Bonnie Wright
val for Paris Hilton
val for Matthias Schoenaerts
val for Tracee Ellis Ross
val for Carrie Underwood
val for Angus T
val for Christina Applegate
val for Cate Blanchett
val for Annie Wersching
val for Jonah Hill
val for Joey Soloway
val for Lea Michele
val for Sharon Stone
val for Scarlett Johansson
val for Bobby Cannavale
val for Halle Berry
val for Carmen Electra
val for Emmy Rossum
val for Jessica Simpson
val for Gerard Butler
val for Aishwarya Rai Bachchan
val for Meagan Good
val for Tony Bennett
val for Bob Hope
val for Jessica Alba
val for Ali Larter
val for Chris Tucker
val for Arjit Taneja
val for Joel Edgerton
val for Kate Mara
val for Jack Nicholson
val for Lady Gaga
val for Cheryl Hines
val for Will Smith
val f

## Face detection

In [14]:
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

In [15]:
def detection(folder, mtcnn):

    data_dir = Path.cwd() / 'data' / f'{folder}_data'
    target_dir = Path.cwd() / 'data' / f'{folder}_data_cropped'

    dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
    dataset.samples = [
        (p, 
         target_dir / Path(p).parents[0].name / Path(p).name)
         for p, _ in dataset.samples
    ]

    loader = DataLoader(
        dataset,
        num_workers=workers,
        batch_size=batch_size,
        collate_fn=training.collate_pil
    )

    for i, (x, y) in enumerate(loader):
        mtcnn(x, save_path=y)
        print(f'Batch {i + 1} of {len(loader)}')


batch_size = 32
epochs = 20
workers = 8

device = torch.device('cuda:0')

mtcnn = MTCNN(
image_size=160, margin=0, min_face_size=20,
thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
device=device
)

detection('train', mtcnn)
detection('val', mtcnn)
detection('test', mtcnn)

# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 1 of 26
Batch 2 of 26
Batch 3 of 26
Batch 4 of 26
Batch 5 of 26
Batch 6 of 26
Batch 7 of 26
Batch 8 of 26
Batch 9 of 26
Batch 10 of 26
Batch 11 of 26
Batch 12 of 26
Batch 13 of 26
Batch 14 of 26
Batch 15 of 26
Batch 16 of 26
Batch 17 of 26
Batch 18 of 26
Batch 19 of 26
Batch 20 of 26
Batch 21 of 26
Batch 22 of 26
Batch 23 of 26
Batch 24 of 26
Batch 25 of 26
Batch 26 of 26
Batch 1 of 5
Batch 2 of 5
Batch 3 of 5
Batch 4 of 5
Batch 5 of 5
Batch 1 of 1


In [16]:
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
train_dataset = datasets.ImageFolder(Path.cwd() / 'data' / 'train_data_cropped', transform=trans)
val_dataset = datasets.ImageFolder(Path.cwd() / 'data' / 'val_data_cropped', transform=trans)

train_loader = DataLoader(
    train_dataset,
    num_workers=workers,
    batch_size=batch_size
)
val_loader = DataLoader(
    val_dataset,
    num_workers=workers,
    batch_size=batch_size
)

resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(train_dataset.class_to_idx)
).to(device)

optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [17]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print(f'\nEpoch {epoch + 1}/{epochs}')
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     4/4    | loss:    6.7171 | fps: 1000.5200 | acc:    0.0000   

Epoch 1/20
----------
Train |    26/26   | loss:    6.8626 | fps:  374.6056 | acc:    0.0000   
Valid |     4/4    | loss:   32.9526 | fps: 1199.0618 | acc:    0.0000   

Epoch 2/20
----------
Train |    26/26   | loss:    6.4969 | fps:  373.6603 | acc:    0.0012   
Valid |     4/4    | loss:    7.1146 | fps: 1171.3794 | acc:    0.0000   

Epoch 3/20
----------
Train |    26/26   | loss:    5.7804 | fps:  369.0317 | acc:    0.0149   
Valid |     4/4    | loss:    7.9866 | fps: 1256.4624 | acc:    0.0000   

Epoch 4/20
----------
Train |    26/26   | loss:    5.1175 | fps:  379.3522 | acc:    0.0608   
Valid |     4/4    | loss:    7.8780 | fps: 1355.4856 | acc:    0.0000   

Epoch 5/20
----------
Train |    26/26   | loss:    4.5633 | fps:  368.9289 | acc:    0.1137   
Valid |     4/4    | loss:    7.7459 | fps: 1227.7445 | acc:    0.0000   

Epoch 6/20
----------
Train |    26/26   | loss: 

### Test

In [24]:
def collate_fn(x):
    return x[0]

test_dataset = datasets.ImageFolder(Path.cwd() / 'data' / 'test_data')
test_dataset.idx_to_class = {i:c for c, i in test_dataset.class_to_idx.items()}
test_loader = DataLoader(test_dataset, collate_fn=collate_fn)

mtcnn = MTCNN(
image_size=160, margin=0, min_face_size=20,
thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
device=device
)

resnet.eval().to(device)

aligned = []
names = []

i = 0
for x, y in test_loader:
    x_aligned = mtcnn(x)
    if x_aligned is not None:
        aligned.append(x_aligned)
        names.append(test_dataset.idx_to_class[y])
        i += 1
    if i == 10:
        break

aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
pd.DataFrame(dists, columns=names, index=names)

,Amber Tamblyn,Miley Cyrus,Miley Cyrus,Sophia Bush,Sophia Bush
Amber Tamblyn,0.000000,36.249657,54.715439,76.937408,64.829414
Miley Cyrus,36.249657,0.000000,67.707031,68.722832,65.746277
Miley Cyrus,54.715439,67.707031,0.000000,78.272041,56.119682
Sophia Bush,76.937408,68.722832,78.272041,0.000000,42.222034
Sophia Bush,64.829414,65.746277,56.119682,42.222034,0.000000
